<h2 style="color: blue;">API Data Wrangling</h2>
<p>This notebook does data wrangling using financia data from the Quandl API. The API token has been removed. The code was made as reusable as possible, while still being specific to a dataset.</p>

In [56]:
import requests
import re
import math
from collections import defaultdict
from statistics import median

In [2]:
#Insert your API here. As long as it's valid, the rest of the code will do the rest. --> API=''
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?{!s}'.format(API)

<h3 style="color: blue;">Extracting 2018 Data</h3>
<p>This function does the data wrangling. It takes an API and queries the Quandl AFX_X database.
    It returns a list of all rows of data from 2018.</p>

In [3]:
def financial_data_by_year(api,*args):
    """Updated to be more flexible. List is passed and returned filled with data from the year passed"""
    url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?{!s}'.format(api)
    jsondict = defaultdict(list)
    data = requests.get(url).json()['dataset_data']
    for arg in args:
        pattern = u'{!s}'.format(arg)
        string = re.compile(pattern)
        for row in data['data']:
            if string.match(row[0]):
                jsondict[arg].append(row)
    return jsondict

<p>This is the instantiation that contains the list of rows from 2018</p>

In [4]:
result=financial_data_by_year(API,2017)

<h3 style="color: blue;">Converting Json to a Dict</h3>
<p>This section turns the json dataset into a python dictionary</p>

In [5]:
def json2dict(api):
    """Function wrapper for financial_data_by_year. It just iterates over a list of years and returns a defaultdict."""
    jsondict=defaultdict(list)
    keys =[year for year in range(2000,2019)]
    jsondict=financial_data_by_year(api, *keys)
    return jsondict

<h3 style="color: blue;">Query Function</h3>
<p> This cell is inserted so the json2dict function doesn't need to be repeatedly called for the following functions.
This function is generalized to answer multiple queries.</p>

In [6]:
dict_data=json2dict(API)

In [47]:
def date_range(year, span=1, interval='days'):
    """Takes a year, a span(interval of average), and a the range of how long to be averaged(day or month).
    For a range of days, if 30 % range != 0, it rounds up the range up to the next even number.
    For months, if 12 % range != 0 it will keep the range until 12/range < 1, 
    after which the range will be the remainder.
    """
    if span == 1 and interval == 'days':
        ave =0
        for row in dict_data[year]:
            ave += row[6]/len(dict_data[year])
        return ave
    elif interval == 'month':
        if span > 12: 
            print("Out of range")
            return None
        start = 0
        diff = 0
        mod = 12 % end 
        end = mod if (start + (12 % end) == 12) else start + span
        while start != 12:
            for mon in range(start,end):
                ave = 0
                try:
                    assert dict_data[year][row][6] > 0
                    ave += dict_data[year][row][6]/len(dict_data[year])
                except AssertionError:
                    pass
            if ave > diff: diff = ave
            start += span
        return diff
        
            
def financial_query(api,query,*years,**kwargs):
    """Open price is the 2nd element of the row data. Skips over empty rows. Edited to be more flexible.
    Now allows multiple queries, and a range to compare dates """
    if query.lower().strip() == 'extremes':
        high=0
        low=0
        for year in years:
            for row in dict_data[year]:
                try:
                    assert type(row[1]) != type(None)
                    if row[1] > high : high = row[1]
                    elif row[1] <= low: low = row[1]
                except AssertionError: pass
        return high, low
    elif query.lower().strip() == 'change':
        for year in years:
            return date_range(year,1)
    elif query.lower().strip() == 'difference':
        change = 0 
        for year in years:
            for row in dict_data[year]:
                index = 0
                high_1st = row[4]
                for index in range(dict_data[year].index(row), len(dict_data[year])):
                    high_2nd = dict_data[year][index][4]
                    change = abs(high_2nd-high_1st) if abs(high_2nd - high_1st) > change else change
        return change
    elif query.lower().strip() == 'range':
        if kwargs.keys() in ['span','interval']:
            for year in years:
                return date_range(year, **kwargs.values())
    elif query.lower().strip() == 'average':
        if kwargs.keys() in ['span','interval']:
            for year in years:
                return date_range(year, **kwargs.values())
        else:
            for year in years:
                return date_range(year)
    elif query.lower().strip() == 'median':
        for year in years:
            trading_vol=[]
            for row in dict_data[year]:
                trading_vol.append(row[6])
                return median(trading_vol)

<h3 style="color: blue;">Finding Highest and Lowest Opening Prices</h3>
<p>Here the highest and lowest opening stock opening prices are calculated. The functionality is added 
to the financial_query function.</p>

In [31]:
high,low = financial_query(API,'extremes',2017)

<h3 style="color: blue;">Finding Largest Fluctuation (One Day)</h3>
<p>Here the greatest difference in opening and closing prices are calculated. The functionality is added 
to the financial_query function.</p>

In [37]:
diff= financial_query(API,'change',2017)

<h3 style="color: blue;">Finding Largest Fluctuation (Two Days)</h3>
<p>Here the greatest difference in opening and closing prices between any two days are calculated.
The functionality is added to the financial_query function.</p>

In [39]:
fluctuation = financial_query(API, 'difference ',2017)

<h3 style="color: blue;">Finding the Daily Average</h3>
<p>The average of daily volume for the year is taken. The function was also generalized to allow search by year, and for different ranges <br>(average by month or year). The functionality is added 
to the financial_query function.</p>

In [48]:
average = financial_query(API,'average',2017)

<h3 style="color: blue;"><i>Optional</i>: Finding the Median For the Year</h3>
<p>This section returns the median value for the year.</p>

In [ ]:
trading_median = financial_query(API,'average',2017)